# **Classificação de Imagens de Frutas usando CNN**

Neste projeto, temos um sistema para classificar imagens em três categorias: maçã, banana ou laranja.Utilizaremos uma Rede Neural Convolucional (CNN) para essa tarefa, pois ela é muito eficaz para extrair características visuais importantes das imagens.

Antes de iniciar o treinamento do modelo, você precisará enviar manualmente os arquivos compactados do dataset filtrado "fruits_filtered.zip" e a pasta "test_images" para o Colab. Esses arquivos contem apenas as pastas das três classes de frutas desejadas e estão com imagens separadas para treinamento e teste, achei que assim tornaria mais fácil de ser testado, pois o dataset original que utilizei para a realização da oficina era muito grande porque haviam diversas classes. A fonte do dataset original utilizado: https://www.kaggle.com/datasets/moltean/fruits?resource=download. Vou deixar instruções no codigo






In [ ]:
from google.colab import files
import zipfile
import os

# Faz upload dos dois arquivos ZIP de uma vez:
uploaded = files.upload()
# O professor selecionará os arquivos "fruits_filtered.zip" e "test_images.zip"

# Extraindo o dataset filtrado (fruits_filtered.zip)
zip_path_dataset = 'fruits_filtered.zip'
extract_path_dataset = 'fruits_filtered'
if not os.path.exists(extract_path_dataset):
    with zipfile.ZipFile(zip_path_dataset, 'r') as zip_ref:
        zip_ref.extractall(extract_path_dataset)
    print("Dataset filtrado extraído com sucesso!")
else:
    print("O dataset filtrado já está extraído.")

# Extraindo as imagens de teste (test_images.zip)
zip_path_test = 'test_images.zip'
extract_path_test = 'test_images'
if not os.path.exists(extract_path_test):
    with zipfile.ZipFile(zip_path_test, 'r') as zip_ref:
        zip_ref.extractall(extract_path_test)
    print("Pasta de teste extraída com sucesso!")
else:
    print("A pasta de teste já está extraída.")


# **1 - Indicação clara do tipo de rede neural usada e justificativa do porquê de ser a escolha mais adequada;**

Resposta:

Utilizaremos uma Rede Neural Convolucional (CNN) para este projeto. Como as CNNs são especialmente projetadas para tarefas de visão computacional, pois elas conseguem identificar automaticamente padrões e características visuais em imagens, como bordas, texturas, formas e núcleos. Essa capacidade de extração de características permite que a rede destaque os elementos essenciais para diferenciar entre as classes de frutas (maçã, banana e laranja). As camadas de pooling das CNNs ajudam a reduzir a dimensionalidade dos dados sem perder informações importantes, o que torna o modelo mais eficiente e menos suscetível ao overfitting. Por todas essas razões, a CNN é a escolha ideal para resolver o problema de classificação proposta no trabalho.


In [8]:
# Importação das bibliotecas
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np

# **2 - Descrição do processo de pré-processamento das imagens**

Resposta:

No pré-processamento, as imagens passam por três etapas principais:

Redimensionamento: É feito um ajuste para que todas as imagens tenham o tamanho de 128×128 pixels, mantendo um padrão.

Normalização: Os valores dos pixels são divididos por 255, transformando-os para a faixa [0, 1]. Isso ajuda o modelo a processar os dados de forma mais eficiente.

Data Augmentation: São aplicadas transformações como rotação, zoom, translação e inversão horizontal para aumentar a diversidade dos dados e reduzir o overfitting, garantindo que o modelo generalize bem.

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Configurando o gerador de dados com normalização e data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,           # Normalização para [0,1]
    validation_split=0.2,     # Reserva 20% dos dados para validação para manter o padrão 80/20
    rotation_range=20,
    zoom_range=0.15,
    horizontal_flip=True
)

# Carrega as imagens do dataset filtrado extraído
train_data = datagen.flow_from_directory(
    extract_path_dataset,      # Caminho para a pasta "fruits_filtered"
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_data = datagen.flow_from_directory(
    extract_path_dataset,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 7592 images belonging to 3 classes.
Found 1896 images belonging to 3 classes.


# **3 - Apresentação do modelo de rede neural em Python, com uma breve explicação de cada camada**

Resposta:

O modelo é uma CNN constituída dos seguintes blocos:

Conv2D: Essa camada analisa a imagem e extrai detalhes essenciais, como bordas e texturas.

MaxPooling2D: Em seguida, a camada de pooling reduz a quantidade de dados, mantendo as características mais importantes.

Flatten: Transforma as informações extraídas em uma lista única (vetor unidimensional), que pode ser usada para a classificação.

Dense: Camadas totalmente conectadas que, com base no vetor, realizam a classificação final.

Dropout: Uma camada que ajuda a evitar o overfitting, desligando aleatoriamente alguns neurônios durante o treinamento.

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Construindo o modelo CNN
model = Sequential()

# Primeira camada convolucional com 32 filtros (3x3) e pooling
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D((2, 2)))

# Segunda camada convolucional com 64 filtros e pooling
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Terceira camada convolucional com 128 filtros e pooling
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Convertendo os mapas de características em um vetor unidimensional
model.add(Flatten())

# Camada densa com 128 neurônios e dropout para evitar overfitting
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

# Camada de saída: 3 neurônios (apple, banana, orange) com softmax para classificar
model.add(Dense(3, activation='softmax'))

# Mostrando um resumo da arquitetura do modelo
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 3)                   │             387 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,305,027 (12.61 MB)

 Trainable params: 3,305,027 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

# **4 - Compilação e treinamento do modelo, mencionando a função de perda e o otimizador usados**

Resposta:

O modelo é compilado com o otimizador adam que é muito usado por sua eficiência e rapidez e a função de perda categorical_crossentropy, adequada para problemas com várias classes. Em seguida, o treinamento é realizado usando os dados de treinamento e validação, permitindo acompanhar a evolução do modelo ao longo das épocas.

In [11]:
# Compilando o modelo
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Treinamento do modelo
history = model.fit(
    train_data,
    epochs=20,        # Número de épocas (pode ser alterado conforme necessário)
    validation_data=validation_data
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
238/238 ━━━━━━━━━━━━━━━━━━━━ 160s 665ms/step - accuracy: 0.9172 - loss: 0.2354 - val_accuracy: 1.0000 - val_loss: 0.0012
Epoch 2/20
238/238 ━━━━━━━━━━━━━━━━━━━━ 160s 671ms/step - accuracy: 0.9983 - loss: 0.0060 - val_accuracy: 0.9989 - val_loss: 0.0019
Epoch 3/20
238/238 ━━━━━━━━━━━━━━━━━━━━ 155s 652ms/step - accuracy: 0.9967 - loss: 0.0108 - val_accuracy: 0.9931 - val_loss: 0.0129
Epoch 4/20
238/238 ━━━━━━━━━━━━━━━━━━━━ 158s 664ms/step - accuracy: 0.9976 - loss: 0.0055 - val_accuracy: 1.0000 - val_loss: 7.9720e-04
Epoch 5/20
238/238 ━━━━━━━━━━━━━━━━━━━━ 161s 677ms/step - accuracy: 0.9989 - loss: 0.0021 - val_accuracy: 0.9968 - val_loss: 0.0179
Epoch 6/20
238/238 ━━━━━━━━━━━━━━━━━━━━ 155s 651ms/step - accuracy: 0.9996 - loss: 0.0024 - val_accuracy: 0.9979 - val_loss: 0.0075
Epoch 7/20
238/238 ━━━━━━━━━━━━━━━━━━━━ 157s 660ms/step - accuracy: 1.0000 - loss: 3.0601e-04 - val_accuracy: 1.0000 - val_loss: 5.8909e-05
Epoch 8/20
238/238 ━━━━━━━━━━━━━━━━━━━━ 159s 670ms/step - accura

# **5 - Processo de avaliação e validação do modelo**

Resposta:

Após o treinamento, o desempenho do modelo é avaliado com o conjunto de validação. A função evaluate retorna as métricas de loss e acurácia, permitindo verificar se o modelo está generalizando bem para novos dados.



In [12]:
# Avaliando o modelo com os dados de validação
loss, accuracy = model.evaluate(validation_data)
print(f"Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

60/60 ━━━━━━━━━━━━━━━━━━━━ 14s 223ms/step - accuracy: 0.9994 - loss: 5.9495e-04
Loss: 0.0005, Accuracy: 0.9995


# **6 - Conclusão explicando como o modelo resolverá o problema proposto**

Resposta:

A CNN criada consegue reconhecer automaticamente os detalhes que diferenciam maçãs, bananas e laranjas. Quando uma nova imagem é enviada, o sistema redimensiona, normaliza e processa a imagem para extrair suas características importantes. Com base nisso, a camada de saída calcula a probabilidade para cada classe, permitindo a classificação automática e precisa da imagem.

# **Teste Interativo – Avaliando uma imagem inédita**

Instrução:
Para testar o sistema com uma imagem que o modelo nunca viu, existem duas opções:

Opção A: Selecionar uma imagem presente na pasta "test_images" e renomear 'img_path' com o caminho do arquivo que deseja testar. Exemplo: test_images/apple/186_100.jpg

Opção B: Fazer upload manual de uma nova imagem.

In [ ]:
# Exemplo opção A
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Definindo o caminho para a imagem (certifique-se de que o arquivo exista)
img_path = 'test_images/pasta(apple ou banana ou orange)/nome_do_arquivo.jpg'
# Substitua indicando o caminho correto da imagem a ser testada

# Carrega a imagem com o tamanho esperado (128x128 pixels)
test_img = load_img(img_path, target_size=(128, 128))
test_img_array = img_to_array(test_img)
test_img_array = np.expand_dims(test_img_array, axis=0)
test_img_array /= 255.0  # Normalização

# Obtém o mapeamento dos índices para os nomes das classes a partir dos dados de treino
class_indices = train_data.class_indices
reverse_class_indices = {v: k for k, v in class_indices.items()}

# Realiza a predição
prediction = model.predict(test_img_array)
predicted_class_index = np.argmax(prediction, axis=1)[0]
predicted_class_name = reverse_class_indices[predicted_class_index]

print("Probabilidades para cada classe:", prediction)
print("Classe predita (índice):", predicted_class_index)
print("Classe predita (nome):", predicted_class_name)

In [13]:
# Exemplo opção B
from google.colab import files
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Obtém o mapeamento dos índices para os nomes das classes
class_indices = train_data.class_indices
reverse_class_indices = dict((v, k) for k, v in class_indices.items())

# Abre uma janela para fazer o upload de uma nova imagem
uploaded_img = files.upload()

# Seleciona a imagem carregada (assume-se que apenas uma imagem foi enviada)
img_name = list(uploaded_img.keys())[0]
print("Imagem carregada:", img_name)

# Carrega e pré-processa a imagem
test_img = load_img(img_name, target_size=(128, 128))
test_img_array = img_to_array(test_img)
test_img_array = np.expand_dims(test_img_array, axis=0)
test_img_array /= 255.0  # Normalização

# Realiza a predição
prediction = model.predict(test_img_array)
predicted_class_index = np.argmax(prediction, axis=1)[0]
predicted_class_name = reverse_class_indices[predicted_class_index]

print("Probabilidades:", prediction)
print("Classe predita (índice):", predicted_class_index)
print("Classe predita (nome):", predicted_class_name)

Saving r_327_100.jpg to r_327_100.jpg
Imagem carregada: r_327_100.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
Probabilidades: [[1.4410609e-06 1.3696231e-15 9.9999857e-01]]
Classe predita (índice): 2
Classe predita (nome): orange
